# Library 정보 수집 

In [17]:
import requests
from enum import Enum
import pandas as pd
from pandas.io.json import json_normalize
import xmltodict
import json
import cx_Oracle
from datetime import datetime
import schedule
import time
from stylecloud import gen_stylecloud
import os

myKey = 'af2df55209c61c0726e45405de83247e14d2474c9c0bca948854d5495b6465a1'
isbnList = []

class LibCode(Enum):
    Code01 = 124004
    Code02 = 124012
    Code03 = 129224
    Code04 = 129219
    Code05 = 124007
    

def initDB():
    # 최신데이터로 업데이트하기 위하여 기존 데이터 삭제
    connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
    cursor = connect.cursor()
    sql_delete = 'delete from book'
    cursor.execute(sql_delete)
    cursor.close()
    connect.commit()
    connect.close()
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 초기화 완료.....')
    
def collectionBook():
    global isbnList
    today_year = str(datetime.today().year)
    today_month = str(datetime.today().month)
    oriUrl = 'http://data4library.kr/api/loanItemSrchByLib'
    region = '24'  #광주
    dtl_region = '24030'  #광주 남구
    pageSize = '2'
    today_set = '&startDt='+today_year+'-'+today_month+'-01&endDt='+today_year+'-'+today_month+'-31'
    count = 0
    checkDB = False
    if isbnList:
        isbnList = []
    for code in LibCode:
        url = oriUrl + "?authKey=" + myKey + today_set + \
            '&pageNo=1&pageSize=' + pageSize + "&libCode=" + str(code.value)
        response = requests.get(url)
        
        # XML 수집
        content = response.text
        
        # XML -> JSON 변환
        result = xmltodict.parse(content)
        dictionay = json.loads(json.dumps(result))
        if dictionay != "" and checkDB == False:
            initDB()
            checkDB = True
        elif checkDB == True:
            pass
        else:
            print('[' + datetime.now().strftime('%Y%m%d') + ']','데이터 수집 오류 발생.....')
            return
        connect = cx_Oracle.connect("Library_admin", "12345", "localhost:1521/xe")
        cursor = connect.cursor()
        df = pd.json_normalize(dictionay['response']['docs']['doc'])
        columns = ['bookname','authors','isbn13','bookImageURL']
        df = df[columns]
        isbnList += df['isbn13'].values.tolist();
        visited = set()
        dup = [x for x in isbnList if x in visited or (visited.add(x) or False)]
        for i in df.index:
            check = False;
            sql_insert = 'INSERT INTO BOOK(book_number,bookname,authors,isbn13,book_ImageURL)' +\
            'VALUES(:booknumber, :bookname, :authors, :isbn13, :bookImageURL)'
            
            booknumber = count
            bookname = df.loc[i].bookname
            authors = df.loc[i].authors
            isbn13 = df.loc[i].isbn13
            bookImageURL = df.loc[i].bookImageURL
            
            # 중복 ISBN 삭제
            for txt in dup:
                result = isbn13.find(txt)
                if(result == -1):
                    check = False
                else:
                    dup.remove(txt)
                    check = True
            
            if check == True:
                continue
            
            cursor.execute(sql_insert, (booknumber, bookname, authors, isbn13, bookImageURL))
            connect.commit()
            count += 1
        connect.close()
    
    print('[' + datetime.now().strftime('%Y%m%d') + ']','DB 데이터 입력 완료.....')
    
    
def bookKeyWord():
    global isbnList
    main_dir_path = "C:/BookLibrary"
    oriUrl = 'http://data4library.kr/api/keywordList'
    path = 'C:\Windows\Fonts\HMFMPYUN.TTF'
    
    if not os.path.isdir(main_dir_path):
        print('폴더를 생성합니다.')
        os.makedirs(main_dir_path)
            
    
    if os.path.exists(main_dir_path):
        print('기존 이미지를 삭제합니다.')
        for file in os.scandir(main_dir_path):
            os.remove(file.path)
    else:
        print('폴더가 존재하지 않습니다.')
    
    isbnList = list(set(isbnList))
    for isbn in isbnList:
        url = oriUrl + "?authKey=" + myKey + "&isbn13=" + isbn + "&format=json"
        response = requests.get(url)
        content = response.json()
        if content['response']['resultNum'] < 50:
            print('isbn : ' + str(isbn) +' / 데이터가 존재하지 않습니다.')
            fileName = 'C:/BookLibrary/' + isbn + '.png'
            gen_stylecloud(text="이미지",
                          font_path=path,
                          icon_name="fas fa-square",
                          palette="colorbrewer.sequential.YlOrBr_3",
                          background_color='black',
                          gradient="horizontal",
                          output_name=fileName)
            continue
        df = pd.json_normalize(content['response']['items'])
        df["item.weight"] = df["item.weight"].astype(int)
        wc_data = df.set_index("item.word").to_dict()["item.weight"]
        
        
        fileName = 'C:/BookLibrary/' + isbn + '.png'
        gen_stylecloud(text=wc_data,
                          font_path=path,
                          icon_name="fas fa-square",
                          palette="colorbrewer.sequential.YlOrBr_3",
                          background_color='black',
                          gradient="horizontal",
                          output_name=fileName)
    print('[' + datetime.now().strftime('%Y%m%d') + ']','이미지 처리 완료.....')
    
    
collectionBook()
bookKeyWord()

# 매주 월요일 오후 13시 30분 업데이트
# schedule.every().monday.at("13:30").do(collectionBook)
# schedule.every().monday.at("13:30").do(bookKeyWord)
# while True:
#     schedule.run_pending()
#     time.sleep(1)

[20221209] DB 초기화 완료.....
[20221209] DB 데이터 입력 완료.....
기존 이미지를 삭제합니다.
[20221209] 이미지 처리 완료.....
